# HySwash: A hybrid method for nearshore wave processes

In [ ]:
import os
import os.path as op

import numpy as np
import xarray as xr

from bluemath_tk.core.io import load_model

import sys

sys.path.insert(0, "..")
from utils.plotting import show_graph_for_different_parameters

root_dir = os.getcwd()
output_dir = op.join(root_dir, "output_Veggy_Hs_mono")
templates_dir = op.join(root_dir, "templates")
export_dir = op.join(root_dir, "exported_Veggy_Hs_mono")
os.makedirs(export_dir, exist_ok=True)

os.environ["OMP_NUM_THREADS"] = "1"

If you want to run this notebook using the pickle files exported in Part 1, execute the code below. Otherwise, you can skip it. Sometimes the download hangs. If it happens, run the cell again

In [2]:
import requests
import tarfile

url = "https://geoocean.sci.unican.es/data/exported_Veggy_Hs_mono.tar.gz"
response = requests.get(url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path=root_dir)

/tmp/ipykernel_15740/2830324098.py:7: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  file.extractall(path=root_dir)


Load swash_model and mda from a pickle file and postprocessed_output NetCDF file

In [3]:
mda_model = load_model(op.join(export_dir, "mda_model.pkl"))
swash_model = load_model(op.join(export_dir, "swash_model.pkl"))
depth_file = op.join(templates_dir, "depth.bot")
depth_array = np.loadtxt(depth_file)
postprocessed_output = xr.open_dataset(
    op.join(export_dir, "output_postprocessed_clean.nc")
)

## 5. Reconstruction: Principal Component Analysis (PCA) & Radial Basis Fucntions (RBF)

The reconstruction of the time series of wave parameters in the position of the buoy is carried out by an interpolation technique based on radial basis functions (RBF), a scheme wich is very convenient for scatter and multivariate data. The RBF approximation has been applied successfully in many fields, usually with better results than other interpolation methods (Hardy, 1990).
    
Suppose that $f=f(x)$ is the real-valued function that we want to approximate. We are given M scattered data points $\{x_1,..., x_M\}$ of dimension $\textit{n}$ and the associated real function values $\{f_1, ..., f_M\}$, being $f_i = f(x_j), j = 1,...,M$. The RBF interpolation method consists of a weighted sum of radially symmetric basic functions located at the data points. The approximation function is assumed to be of the form:
$$RBF(x) = p(x) + \sum\limits_{j=1}^M a_j\Phi{\large (}{\large \parallel}{x - x_j}{\large \parallel}{\large )}$$

### Hs reconstruction

In [5]:
from bluemath_tk.datamining.pca import PCA
from bluemath_tk.interpolation.rbf import RBF

postprocessed_output = postprocessed_output.copy(deep=True)

# Apply PCA to the postprocessed output
pca = PCA()
_pcs_ds = pca.fit_transform(
    data=postprocessed_output,
    vars_to_stack=["Hs"],
    coords_to_stack=["Xp"],
    pca_dim_for_rows="case_num",
    value_to_replace_nans={"Hs": 0.0},
)

# Apply RBF reconstruction
rbf = RBF()
rbf.fit(
    subset_data=mda_model.centroids.iloc[postprocessed_output["case_num"].values, :],
    target_data=pca.pcs_df,
)

pca.save_model(
    model_path=op.join(export_dir, "pca_model.pkl"),
)
rbf.save_model(
    model_path=op.join(export_dir, "rbf_model.pkl"),
)


        -------------------------------------------------------------------
        | Initializing PCA reduction model with the following parameters:
        |    - n_components: 0.98
        |    - is_incremental: False
        | For more information, please refer to the documentation.
        -------------------------------------------------------------------
        

        ---------------------------------------------------------------------------------
        | Initializing RBF interpolation model with the following parameters:
        |    - sigma_min: 0.001
        |    - sigma_max: 0.1
        |    - sigma_diff: 0.0001
        |    - sigma_opt: None
        |    - kernel: gaussian
        |    - smooth: 1e-05
        | For more information, please refer to the documentation.
        | Recommended lecture: https://link.springer.com/article/10.1023/A:1018975909870
        ---------------------------------------------------------------------------------
        


KeyboardInterrupt: 

The process PCA + RBF takes 2 hours. To avoid run them again we can load the models from the exported folder

In [ ]:
pca = load_model(op.join(export_dir, "pca_model.pkl"))
rbf = load_model(op.join(export_dir, "rbf_model.pkl"))

Visualize the effect of the plants on Hs

In [8]:
variables_to_analyse_in_metamodel = ["Hs", "Hs_L0", "Wv", "hv", "Nv"]
lhs_parameters = {
    "num_dimensions": 5,
    "num_samples": 11000,
    "dimensions_names": variables_to_analyse_in_metamodel,
    "lower_bounds": [0.5, 0.003, 0, 0, 0],
    "upper_bounds": [3, 0.01, 200, 1.5, 1000],
}

show_graph_for_different_parameters(
    pca=pca, rbf=rbf, lhs_parameters=lhs_parameters, depthfile=depth_file
)

interactive(children=(FloatSlider(value=1.4363502971184063, description='Hs', max=3.0, min=0.5, step=0.25), Fl…

<function utils.plotting.show_graph_for_different_parameters.<locals>.update_plot(Hs, Hs_L0, Wv, hv, Nv)>